# TF acs-2 screen

In [2]:
#%matplotlib inline
import os
import sys
import tifffile as tiff
import pandas as pd
import matplotlib
# For visualising
#matplotlib.use('MacOSX') 
# For saving
#matplotlib.use('Agg') 
matplotlib.use('MacOSX') #TkAgg
import matplotlib.pyplot as plt

import numpy as np
import itertools as IT
import time
import glob
import skimage
from skimage import io as skio
from skimage import color

import cPickle
import gzip

import datetime
from multiprocessing import Pool, Manager
from multiprocessing.sharedctypes import Value, Array
from ctypes import Structure, c_double, c_int

sys.path.append("./src")
from utilities import *

#plt.interactive(True)
matplotlib.rcParams.update({'font.size': 12})
np.set_printoptions(precision=3)



## Set working directory

In [3]:
os.chdir("/Users/Povilas/Dropbox/Projects/Metformin_TF_acs-2/")
sourceloc = "Users/Povilas/Dropbox/Projects/Metformin_TF_acs-2/"

# os.chdir("/home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/")
# sourceloc = "home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/"

odir="."

## Collect info about files

In [8]:
filedata=[]
header=['Replicate_folder','Folder','File']
filedata.append(header)

allfiles=glob.glob('./*/*/*.tiff')

for fl in allfiles:
    rep,fld,fln=fl.split('/')[1:]
    filedata.append([rep,fld,fln])
    #print rep, fld, fln
    
writecsv(filedata,'Allfiles.csv')

## Parse info

In [4]:
#filelist=pd.read_csv('./Allfiles_annotated.csv')

oimages="Images.pkl"

#oimagesgz="Images.pkl.gz"

## Collect images

In [ ]:
settings={'colordepth':'uint8'}
args=[(fl,settings,) for fl in filesel]

results=multiprocessor(4,collector_M,args)
fls,imagesarray=zip(*results)

imagesindex={fl:fli for fli,fl in enumerate(fls) }

print("Saving images: {}".format(time.ctime()))
print "Stacking..."
images=np.stack(imagesarray, axis=0)
print "Done!"

f = open(oimages, "w")
#f = gzip.open(oimagesgz, "w")
print "Writing..."
cPickle.dump((imagesindex,images), f)
f.close()
os.system("gzip {}".format(oimages))
print("Saving images complete!")



## Load images

In [5]:
iimagesgz="Images.pkl.gz"
print time.ctime()
print "Loading images... {}".format(time.ctime())
idata = gzip.open(iimagesgz,'rb')
imagesindex, images = cPickle.load(idata)
idata.close()
print "Loading finished! {}".format(time.ctime())
print time.ctime()


Sat Mar  3 22:19:37 2018
Loading images... Sat Mar  3 22:19:37 2018
Loading finished! Sat Mar  3 22:20:31 2018
Sat Mar  3 22:20:31 2018


## Segment images

In [7]:
settings={'fd':10}

args=[(fln,images[imagesindex[fln]],settings) for fln in imagesindex.keys()]

results=multiprocessor(4,labeller5_M,args)

fls, filteredarray, labelsarray, resultsarray = zip(*results)

#Collect and save results
labelsindex={fl:fli for fli,fl in enumerate(fls) }

results_data=[]
res_header=['File','Worm','W_N','W_Sum','W_Mean','B_N','B_Sum','B_Mean']

for fl, respack in zip(fls, resultsarray):
    for row in respack:
        if len(row)>0:
            results_data.append([fl]+row)
            

# Save results
results=pd.DataFrame(results_data,columns=res_header)
results.to_csv(otbl)

print("Saving labels: {}".format(time.ctime()))
print "Stacking..."

labels=np.stack(labelsarray, axis=0)
filtered=np.stack(filteredarray, axis=0)
print "Stacking complete!"

print "Writing..."
#f = gzip.open(olabelsgz, "w")
f = open(olabels, "w")
#cPickle.dump((labelsindex,labels), f)
cPickle.dump((labelsindex,filtered,labels), f)
f.close()
print "Zipping..."
os.system("gzip -f {}".format(olabels))
print("Saving labels complete! {}".format(time.ctime()))

/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


Done   0%,     5/735   remaining: 0:24:40 total: 0:24:51
Done   1%,     8/735   remaining: 0:22:56 total: 0:23:11
Done   1%,    12/735   remaining: 0:20:13 total: 0:20:33
Done   2%,    16/735   remaining: 0:18:50 total: 0:19:15
Done   2%,    20/735   remaining: 0:17:57 total: 0:18:27
Done   3%,    23/735   remaining: 0:18:08 total: 0:18:43
Done   3%,    27/735   remaining: 0:17:32 total: 0:18:13
Done   4%,    32/735   remaining: 0:16:31 total: 0:17:17
Done   4%,    36/735   remaining: 0:16:13 total: 0:17:04
Done   5%,    40/735   remaining: 0:15:58 total: 0:16:53
Done   5%,    43/735   remaining: 0:16:08 total: 0:17:08
Done   6%,    47/735   remaining: 0:15:53 total: 0:16:58
Done   6%,    51/735   remaining: 0:15:40 total: 0:16:51
Done   7%,    56/735   remaining: 0:15:11 total: 0:16:26
Done   8%,    59/735   remaining: 0:15:18 total: 0:16:38
Done   8%,    63/735   remaining: 0:15:08 total: 0:16:33
Done   9%,    67/735   remaining: 0:14:58 total: 0:16:29
Done   9%,    71/735   remainin

### Load data

In [11]:
fileinfo=readTF('./Allfiles_annotated.csv')
filelist=pd.read_csv('./Allfiles_annotated.csv')
genes=fileinfo.keys()
#genes=['OP50-C']

iimages="Images.pkl.gz"
ilabels="Segmentation_adaptive_fd10.pkl.gz"

ofigs="Segmentation_adaptive/{}_segmentation_adaptive.pdf"

repflds=['Rep1_23-8-17','Rep2_24-8-17']

plt.ioff()

./Allfiles_annotated.csv
Index([u'Replicate', u'Type', u'Gene', u'FileNo', u'FileInd',
       u'Replicate_folder', u'Folder', u'File', u'Max'],
      dtype='object')
Necessary headers found!


In [5]:
print time.ctime()

print "Loading images..."
imagesf = gzip.open(iimages,'rb')
imagesindex, images = cPickle.load(imagesf)
imagesf.close()
print "Loading finished!"
print time.ctime()

print "Loading labels..."
labelsf = gzip.open(ilabels,'rb')
labelspickle = cPickle.load(labelsf)
if len(labelspickle)==3:
    labelsindex, filtered, labels=labelspickle
else:
    labelsindex, labels=labelspickle
labelsf.close()
print "Loading finished!"
print time.ctime()

Sat Mar  3 21:42:47 2018
Loading images...
Loading finished!
Sat Mar  3 21:43:37 2018
Loading labels...
Loading finished!
Sat Mar  3 21:45:30 2018


### Generate figures

In [12]:
#def plotTF_M((gene,ofigs,fileinfo,imagesindex,images,labelsindex,filtered,labels, settings,q)):
    #ofigs,fileinfo, imagesindex, labelsindex, filtered, labels,
    #global
    
#     fileinfo=Afileinfo
#     imagesindex=Aimagesindex
#     images=Aimages
#     labelsindex=Alabelsindex
#     filtered=Afiltered
#     labels=Alabels
for gene in genes:   
    if gene=='OP50-C':
        nflds=6*2
    else:
        nflds=4*2
        
    gval=fileinfo[gene]
    gmax = gval['Max']
    print gmax
    
    fwidth=6*nflds
    fheight=5*gmax
    
    #print nflds, gmax, fwidth, fheight
    fig, axes = plt.subplots(nrows=gmax, ncols=nflds, figsize=(fwidth,fheight), dpi=300)
    ofile=ofigs.format(gene)
    
    #print ofile
    col=0
    for rep in [1,2]:
        repval = gval[rep]
        repfld=repflds[rep-1]
        for tp in ['C','T']:
            tpval=repval[tp]
            for fld, fldvals in tpval.iteritems():
                print "Gene: {} Replicate: {} Type: {} Folder: {}".format(gene, rep, tp, fld)
                fldcontsel=fldvals['ByFIN']
                #print fldcontsel.keys()
                
                row=0
                for fli in fldcontsel.keys():
                    fl=fldcontsel[fli]
                    flfull="./{}/{}/{}".format(repfld,fld,fl)
                    print flfull #, os.path.isfile(flfull)
                    
                    #GetRGB data from a collapsed image
                    image_raw=images[imagesindex[flfull]]
                    if np.ndim(image_raw)==2:
                        #v=image_raw
                        image=color.gray2rgb(image_raw)
                    else:
                        #v=color.rgb2hsv(image)[:,:,2]
                        image=image_raw
                    #Load labels
                    
                    imagef=filtered[labelsindex[flfull]]
                    labeled_worms=labels[labelsindex[flfull]]

                    plotcontours(imagef,axes[row,col],ylabel=fl,title="{} - {}".format(rep,fld),plotcontour=False,plotlabels=False,white=False)
                    plotcontours(image,axes[row,col+1],labeled_worms,title="{}".format(fli),plotlabels=False)
                    row+=1 
                    
                    #q.put(flfull) 
                col+=2

           
    #fig.tight_layout()
    fig.savefig(ofile, bbox_inches='tight')
    plt.close(fig)

print "Finished!"

18
Gene: mlc Replicate: 1 Type: C Folder: mlc wo metf
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF1.tiff


/usr/local/lib/python2.7/site-packages/matplotlib/artist.py:210: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)


./Rep1_23-8-17/mlc wo metf/mlc wo metf TF2.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF3.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF4.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF5.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF6.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF7.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF8.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF9.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF10.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF11.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF12.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF13.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF14.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF15.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF16.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF17.tiff
Gene: mlc Replicate: 1 Type: T Folder: mlc metf
./Rep1_23-8-17/mlc metf/Original Image1.tiff
./Rep1_23-8-17/mlc metf/captured layer 32.tiff
./Rep1_23-8-17/mlc metf/captured layer 43.tiff
./Rep1_23-8-17/mlc metf/captured laye

In [22]:
Aimages=Array(c_double,images.shape)
Aimages=images.copy()

Alabels=Array(c_double,labels.shape)
Alabels=labels.copy()

Afiltered=Array(c_double,filtered.shape)
Afiltered=filtered.copy()

m = Manager()
Afileinfo=m.dict()
Afileinfo.update(fileinfo)

Aimagesindex=m.dict()
Aimagesindex.update(imagesindex)

Alabelsindex=m.dict()
Alabelsindex.update(labelsindex)

settings={}

#global fileinfo, imagesindex, images, labelsindex,filtered, labels

#global Afileinfo,Aimagesindex,Aimages,Alabelsindex,Afiltered,Alabels

args=[(gene,ofigs,Afileinfo,Aimagesindex,Aimages,Alabelsindex,Afiltered,Alabels, settings) for gene in genes]

results=multiprocessor(4,plotTF_M,args,filelist.shape[0])
